In [ ]:
##ここから

In [ ]:
mkdir dogs-vs-cats

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip') as existing_zip:
    existing_zip.extractall('./dogs-vs-cats')

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip') as existing_zip:
    existing_zip.extractall('./dogs-vs-cats')

In [ ]:
cd dogs-vs-cats/train

In [ ]:
mkdir cats

In [ ]:
mkdir dogs

In [ ]:
import shutil
import glob
import os

def move_glob(dst_path, pathname, recursive=True):
    for p in glob.glob(pathname, recursive=recursive):
        shutil.move(p, dst_path)

In [ ]:
move_glob('./dogs', 'dog*.jpg')

In [ ]:
move_glob('./cats', 'cat*.jpg')

In [ ]:
cd ../

In [ ]:
mkdir valid

In [ ]:
cd valid

In [ ]:
mkdir dogs

In [ ]:
mkdir cats

In [ ]:
cd ../

In [ ]:
cd ../

In [ ]:
import cv2
import numpy as np

def resizehalf(src_file, dst_file):
    src_img = cv2.imread(src_file)
    h, w, c = src_img.shape

    #比率設定
    scale_w = 1 / 2
    scale_h = 1 / 2

    resize_img = cv2.resize(src_img, dsize=None, fx=scale_w, fy=scale_w, interpolation = cv2.INTER_AREA)

    # dst_img 生成
    dst_img = np.zeros((h//2 + h%2, w//2 + w%2, 3), dtype = np.uint8)

    # dst_imgにresize_imgを合成
    h, w, c = resize_img.shape
    dst_img[0:h, 0:w] = resize_img

    cv2.imwrite(dst_file, dst_img)

In [ ]:
for num in range(0, 12500):
    resizehalf('./dogs-vs-cats/train/dogs/dog.' + str(num) + '.jpg', './dogs-vs-cats/train/dogs/dog.' + str(num+12500) + '.jpg')
    resizehalf('./dogs-vs-cats/train/cats/cat.' + str(num) + '.jpg', './dogs-vs-cats/train/cats/cat.' + str(num+12500) + '.jpg')

In [ ]:
import cv2
import numpy as np

def resize(src_file, dst_file, width, height):
    """
    画像ファイルを与えられたサイズにサイズ変換する関数
    アス比は固定、左上に寄せる、余った部分はゼロで埋める

    Returns
    -------
    ret_scale : float
    拡大/縮小した倍率
    """

    src_img = cv2.imread(src_file)
    h, w, c = src_img.shape
    # dst_img = cv2.resize(src_img, dsize=(width, height))

    # アス比固定, padding
    scale_w = width / w
    scale_h = height / h

    ret_scale = 1.0
    # Down Convert
    if(scale_w < 1.0 or scale_h < 1.0):
        if(scale_w < scale_h): 
            resize_img = cv2.resize(src_img, dsize=None, fx=scale_w, fy=scale_w, interpolation = cv2.INTER_AREA)
            ret_scale = scale_w
        else:
            resize_img = cv2.resize(src_img, dsize=None, fx=scale_h, fy=scale_h, interpolation = cv2.INTER_AREA)
            ret_scale = scale_h
    else:
        resize_img = src_img

    # dst_img 生成
    dst_img = np.zeros((height, width, 3), dtype = np.uint8)

    # dst_imgにresize_imgを合成
    top = 0
    left = 0
    #dst_img[top:height + top, left:width + left] = resize_img
    h, w, c = resize_img.shape
    dst_img[0:h, 0:w] = resize_img

    cv2.imwrite(dst_file, dst_img)

    return ret_scale

In [ ]:
for num in range(0, 12500):
    resize('./dogs-vs-cats/train/dogs/dog.' + str(num) + '.jpg', './dogs-vs-cats/train/dogs/dog.' + str(num) + '.jpg', 216, 216)
    resize('./dogs-vs-cats/train/cats/cat.' + str(num) + '.jpg', './dogs-vs-cats/train/cats/cat.' + str(num) + '.jpg', 216, 216)

In [ ]:
for num in range(5000):
    move_glob('./dogs-vs-cats/valid/cats', './dogs-vs-cats/train/cats/cat.'+ str(num) +'.jpg')
    move_glob('./dogs-vs-cats/valid/dogs', './dogs-vs-cats/train/dogs/dog.'+ str(num) +'.jpg')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_dir = './dogs-vs-cats/train'
valid_data_dir = './dogs-vs-cats/valid'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
valid_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(directory = train_data_dir,target_size = (216, 216),batch_size = 32, class_mode = "binary")
valid_generator = valid_datagen.flow_from_directory(directory = valid_data_dir,target_size = (216, 216),batch_size = 32, class_mode = "binary")

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.layers import Conv2D,MaxPooling2D,Dropout
from keras import optimizers

In [ ]:
model = Sequential()

model.add(Conv2D(64,kernel_size = (5,5),activation = "relu",input_shape = (216,216,3)))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,kernel_size = (5,5),activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128,kernel_size = (5,5),activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128,kernel_size = (5,5),activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256,activation = "relu"))
model.add(Dropout(0.25))
model.add(Dense(1,activation = "sigmoid"))

sgd = optimizers.SGD(lr = 0.1)

model.compile(loss = "mean_squared_error",optimizer = sgd,metrics = ["accuracy"])
history = model.fit_generator(train_generator,epochs = 30,verbose = 1, validation_data = (valid_generator), steps_per_epoch = 40000 / 32,validation_steps = 10000/32)

In [ ]:
model.save("./dogs-vs-cats/dogs-vs-cats_model.hdf5")

In [ ]:
from keras.utils import plot_model
model.summary()
plot_model(model,to_file='./model1.png')

In [ ]:
import matplotlib.pyplot as plt

loss = history.history["loss"]
val_loss = history.history["val_loss"]

learning_count = len(loss)+1

plt.plot(range(1,learning_count),loss,marker = "+",label = "loss")
plt.plot(range(1,learning_count),val_loss,marker = "+",label = "val_loss")
plt.legend(loc = "best",fontsize = 10)
plt.xlabel("learning_count")
plt.ylabel("loss")
plt.show()

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

learning_count = len(loss)+1

plt.plot(range(1,learning_count),accuracy,marker = "+",label = "accuracy")
plt.plot(range(1,learning_count),val_accuracy,marker = "+",label = "val_accuracy")
plt.legend(loc = "best",fontsize = 10)
plt.xlabel("learning_count")
plt.ylabel("accuracy")
plt.show()

In [ ]:
from keras.models import load_model
new_model = load_model('./dogs-vs-cats/dogs-vs-cats_model.hdf5')

In [ ]:
for num in range(1, 12501):
    resize('./dogs-vs-cats/test/' + str(num) + '.jpg', './dogs-vs-cats/test/' + str(num) + '.jpg', 216, 216)

In [ ]:
result_num = []

for num in range(1, 12501):
    result_num.append(num)

In [ ]:
#100枚までの結果出力
from skimage import io
import numpy as np
from skimage.transform import resize
from keras.preprocessing.image import load_img, img_to_array
result = []

%matplotlib inline

#./train1/cats/cat.
#./test/
for num in range(1, 12501):
    img = io.imread('./dogs-vs-cats/test/'+ str(num) +'.jpg')
    io.imshow(img)

    # 判定する画像の読み込み
    image_infer = load_img('./dogs-vs-cats/test/'+ str(num) +'.jpg', target_size=(216, 216))
    data_infer = img_to_array(image_infer)
    data_infer = np.expand_dims(data_infer, axis=0)
    data_infer = data_infer / 255.0

    # 判定処理の実行
    model.load_weights(os.path.join('./dogs-vs-cats/', 'dogs-vs-cats_model.hdf5'))
    result_predict = model.predict(data_infer)[0] * 100
    result.append(result_predict)
    if num % 100 == 0:
        print(num,'：Dog (%.1f%%)' % result_predict)

    # 判定結果の出力
    #if result[0] > result[1]:
    #    print('Cat (%.1f%%)' % result[0])
    #else:
    #print(num,'：Dog (%.1f%%)' % result_predict)

In [ ]:
import csv
import pandas as pd
df = pd.DataFrame([result])
df = df.T
for col in df:
    df[col]=df[col].astype(str).str.replace("[","").str.replace("]","")

df.to_csv("./sample_submission.csv", index=False)